<a href="https://colab.research.google.com/github/thiagodepaulo/nlp/blob/enap/classificacao_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [ ]:
# prompt: load 20newsgroups dataset from sklearn and convert it in Hugging face Dataset

from sklearn.datasets import fetch_20newsgroups
from datasets import Dataset, DatasetDict, ClassLabel
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModel
import torch

import numpy as np
from sklearn

In [ ]:
categories = [
    "alt.atheism",
    "talk.religion.misc",
    "comp.graphics",
    "sci.space",
]
remove=("headers", "footers", "quotes")

ng = fetch_20newsgroups(subset="all", categories=categories,
                              shuffle=True, random_state=42, remove=remove)

X_train, X_temp, y_train, y_temp = train_test_split(ng.data, ng.target, test_size=0.3, random_state=42)
X_eval, X_test, y_eval, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

hf_dataset_dict = {'train': Dataset.from_dict({"text": X_train, "label": y_train}),
              'eval' : Dataset.from_dict({"text": X_eval, "label": y_eval}),
              'test' : Dataset.from_dict({"text": X_test, "label": y_test}),
}
ng_dataset = DatasetDict(hf_dataset_dict)
ng_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 2370
    })
    eval: Dataset({
        features: ['text', 'label'],
        num_rows: 508
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 509
    })
})

In [ ]:
model_ckpt = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_ckpt, model_max_length=512, padding='max_length', truncation=True)

tokenizer.model_max_length

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

512

In [ ]:
def tokenize(batch):
  return tokenizer(batch['text'], padding=True, truncation=True)

ng_encoded = ng_dataset.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/2370 [00:00<?, ? examples/s]

Map:   0%|          | 0/508 [00:00<?, ? examples/s]

Map:   0%|          | 0/509 [00:00<?, ? examples/s]

In [ ]:
ng_encoded.column_names

{'train': ['text', 'label', 'input_ids', 'attention_mask'],
 'eval': ['text', 'label', 'input_ids', 'attention_mask'],
 'test': ['text', 'label', 'input_ids', 'attention_mask']}

In [ ]:
print(ng_encoded['train'][0]['input_ids'])

[101, 2941, 2065, 2720, 1060, 2018, 2242, 2000, 5114, 2011, 2010, 4447, 2010, 4070, 1997, 1996, 2824, 2052, 13221, 4140, 2022, 1996, 2087, 9768, 1012, 2553, 1998, 2391, 1010, 1996, 15218, 1012, 2011, 6815, 2008, 1996, 15218, 2941, 3047, 1996, 2220, 14512, 2015, 2020, 2583, 2000, 10463, 2116, 2000, 7988, 1012, 2174, 1010, 2065, 2017, 3191, 25436, 2676, 1024, 4229, 1006, 1029, 1007, 1998, 1996, 2553, 2005, 1996, 23053, 6586, 2040, 2939, 2105, 6744, 1998, 2596, 2000, 1000, 2116, 2111, 1000, 2017, 2052, 5382, 2008, 2060, 7862, 1006, 3312, 2271, 2005, 2028, 1007, 2052, 2031, 2988, 2006, 2009, 2035, 2065, 2009, 3047, 1012, 1996, 2755, 2008, 1996, 6331, 8847, 1997, 2824, 1997, 2107, 2307, 10194, 2008, 2027, 2052, 2031, 2042, 4384, 2579, 2007, 1996, 2755, 2008, 2027, 2024, 2025, 2988, 2006, 2011, 7862, 2071, 2069, 2812, 2008, 1996, 6331, 3397, 2116, 2081, 2039, 3441, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
print(tokenizer.convert_ids_to_tokens(ng_encoded['train'][0]['input_ids']))

['[CLS]', 'actually', 'if', 'mr', 'x', 'had', 'something', 'to', 'gain', 'by', 'his', 'claims', 'his', 'account', 'of', 'the', 'events', 'would', 'nm', '##ot', 'be', 'the', 'most', 'respected', '.', 'case', 'and', 'point', ',', 'the', 'resurrection', '.', 'by', 'claiming', 'that', 'the', 'resurrection', 'actually', 'happened', 'the', 'early', 'preacher', '##s', 'were', 'able', 'to', 'convert', 'many', 'to', 'christianity', '.', 'however', ',', 'if', 'you', 'read', 'mathew', '27', ':', '38', '(', '?', ')', 'and', 'the', 'case', 'for', 'the', 'resurrected', 'saints', 'who', 'walked', 'around', 'jerusalem', 'and', 'appeared', 'to', '"', 'many', 'people', '"', 'you', 'would', 'realize', 'that', 'other', 'historians', '(', 'joseph', '##us', 'for', 'one', ')', 'would', 'have', 'reported', 'on', 'it', 'all', 'if', 'it', 'happened', '.', 'the', 'fact', 'that', 'the', 'bible', 'speaks', 'of', 'events', 'of', 'such', 'great', 'magnitude', 'that', 'they', 'would', 'have', 'been', 'noticed', 'take

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

model = AutoModel.from_pretrained(model_ckpt).to(device)

cpu


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
ng_encoded.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

## Remover a última camada do modelo

In [ ]:
text = 'this is a test'
input = tokenizer(text, return_tensors='pt')
input = {k:v.to(device) for k,v in input.items()}

with torch.no_grad():
  outputs = model(**input)
print(outputs)

BaseModelOutput(last_hidden_state=tensor([[[-0.1565, -0.1862,  0.0528,  ..., -0.1188,  0.0662,  0.5470],
         [-0.3575, -0.6484, -0.0618,  ..., -0.3040,  0.3508,  0.5221],
         [-0.2772, -0.4459,  0.1818,  ..., -0.0948, -0.0076,  0.9958],
         [-0.2841, -0.3917,  0.3753,  ..., -0.2151, -0.1173,  1.0526],
         [ 0.2661, -0.5094, -0.3180,  ..., -0.4203,  0.0144, -0.2149],
         [ 0.9441,  0.0112, -0.4714,  ...,  0.1439, -0.7288, -0.1619]]]), hidden_states=None, attentions=None)


In [ ]:
def extract_hidden_states(batch):
  inputs = {k:v.to(device) for k,v in batch.items() if k in tokenizer.model_input_names}
  with torch.no_grad():
    last_hidden_state = model(**inputs).last_hidden_state
  return {'hidden_state':last_hidden_state[:,0].cpu().numpy()}


In [ ]:
ng_hidden = ng_encoded.map(extract_hidden_states, batched=True, batch_size=8)

Map:   0%|          | 0/2370 [00:00<?, ? examples/s]

Map:   0%|          | 0/508 [00:00<?, ? examples/s]

Map:   0%|          | 0/509 [00:00<?, ? examples/s]

In [ ]:
ng_hidden['train'].column_names

['text', 'label', 'input_ids', 'attention_mask', 'hidden_state']

In [ ]:
ng_hidden['train']['hidden_state'].shape

torch.Size([2370, 768])

In [ ]:
X_train = np.array(np_hidden['train']['hidden_state'])
X_eval = np.array(np_hidden['eval']['hidden_state'])
y_train = np.array(np_hidden['train']['label'])
y_eval = np.array(np_hidden['eval']['label'])
X_train.shape, X_eval.shape

NameError: name 'np_hidden' is not defined